In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
from collections import defaultdict
from sklearn.metrics import confusion_matrix
import warnings

from scipy.stats import pearsonr

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, matthews_corrcoef, f1_score
from sklearn.feature_selection import RFE

from skimage.morphology import skeletonize, square, dilation 
from PVBM.GeometricalAnalysis import GeometricalVBMs
from PIL import Image,ImageFilter
from glob import glob

from itertools import combinations

In [2]:
img_size = (300, 300)
batch_size = 50

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(img_size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ColorJitter(brightness=0.5, contrast=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.3998, 0.1676, 0.0636], [0.2762, 0.1356, 0.0666])
    ]),
    'val': transforms.Compose([
        transforms.Resize(img_size),
        transforms.ToTensor(),
        transforms.Normalize([0.3998, 0.1676, 0.0636], [0.2762, 0.1356, 0.0666])
    ]),
    'test': transforms.Compose([
        transforms.Resize(img_size),
        transforms.ToTensor(),
        transforms.Normalize([0.3998, 0.1676, 0.0636], [0.2762, 0.1356, 0.0666])
    ])
}

data_transforms_vessel = {
    'train': transforms.Compose([
        transforms.Resize(img_size),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ToTensor()        
    ]),
    'val': transforms.Compose([
        transforms.Resize(img_size),
        transforms.ToTensor()        
    ]),
    'test': transforms.Compose([
        transforms.Resize(img_size),
        transforms.ToTensor()        
    ])
}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_ft = model_ft = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=False)
num_ftrs = model_ft.classifier.in_features
model_ft.classifier  = nn.Sequential(nn.Linear(num_ftrs, 500),nn.Linear(500,  2))

Using cache found in C:\Users\micha/.cache\torch\hub\pytorch_vision_v0.10.0
c:\Users\micha\anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\micha\anaconda3\envs\pytorch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [3]:
for typ in ['_vessel', '', '_prep']:
    data_dir = './data/data_m'+typ
    if typ == '_vessel': trans = data_transforms_vessel
    else: trans = data_transforms
    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), trans[x]) for x in ['train', 'val', 'test']}
    dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=False, num_workers=4) for x in ['train', 'val', 'test']}

    model_ft.load_state_dict(torch.load('models/densenet121'+typ+'_e20_s300_b14.pt'))
    model_ft.to(device)
    model_ft.eval()

    x, y = {}, {}
    for set in ['train', 'test']:
        for i, data in enumerate(dataloaders[set], 0):
            samples, labels = data
            with torch.no_grad():
                samples = samples.to(device)
                output = model_ft.features(samples)
                output = nn.AdaptiveAvgPool3d(output_size=(1024,1,1))(output).squeeze()
                if i == 0: x[set], y[set] = torch.Tensor.numpy(output.cpu()), torch.Tensor.numpy(labels)
                else:
                    x[set] = np.concatenate((x[set], torch.Tensor.numpy(output.cpu())))
                    y[set] = np.concatenate((y[set], torch.Tensor.numpy(labels)))

    print('FOREST'+typ)                    
    forest = RandomForestClassifier(n_estimators=750, max_depth=6)
    forest.fit(x['train'], y['train'])

    prediction = forest.predict(x['train'])
    tn, fp, fn, tp = confusion_matrix(y['train'], prediction).ravel()
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    recall = tp/(tp+fn)
    precision = tp/(tp+fp)
    specificity = tn/(fp+tn)
    f1_sc = f1_score(y['train'], prediction)
    matthews_cor = matthews_corrcoef(y['train'], prediction)

    print('Training\naccuracy: ', accuracy, '\nrecall', recall, '\nprecision', precision, '\nspecificity', specificity, '\nf1_score', f1_sc, '\nmatthews_cor', matthews_cor)
    
    prediction = forest.predict(x['test'])
    tn, fp, fn, tp = confusion_matrix(y['test'], prediction).ravel()
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    recall = tp/(tp+fn)
    precision = tp/(tp+fp)
    specificity = tn/(fp+tn)
    f1_sc = f1_score(y['test'], prediction)
    matthews_cor = matthews_corrcoef(y['test'], prediction)

    print('Test\naccuracy: ', accuracy, '\nrecall', recall, '\nprecision', precision, '\nspecificity', specificity, '\nf1_score', f1_sc, '\nmatthews_cor', matthews_cor)

    
    sc = StandardScaler()
    x['train']= sc.fit_transform(x['train'])
    x['test'] = sc.transform(x['test'])

    print('REGRESTION'+typ)
    
    reg = LogisticRegression(solver='liblinear', max_iter=1000)
    reg.fit(x['train'], y['train'])
    
    prediction = reg.predict(x['train'])
    tn, fp, fn, tp = confusion_matrix(y['train'], prediction).ravel()
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    recall = tp/(tp+fn)
    precision = tp/(tp+fp)
    specificity = tn/(fp+tn)
    f1_sc = f1_score(y['train'], prediction)
    matthews_cor = matthews_corrcoef(y['train'], prediction)

    print('Training\naccuracy: ', accuracy, '\nrecall', recall, '\nprecision', precision, '\nspecificity', specificity, '\nf1_score', f1_sc, '\nmatthews_cor', matthews_cor)

    prediction = reg.predict(x['test'])
    tn, fp, fn, tp = confusion_matrix(y['test'], prediction).ravel()
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    recall = tp/(tp+fn)
    precision = tp/(tp+fp)
    specificity = tn/(fp+tn)
    f1_sc = f1_score(y['test'], prediction)
    matthews_cor = matthews_corrcoef(y['test'], prediction)

    print('Test\naccuracy: ', accuracy, '\nrecall', recall, '\nprecision', precision, '\nspecificity', specificity, '\nf1_score', f1_sc, '\nmatthews_cor', matthews_cor)
    
    print('SELECTED')
    
    sfs1 = RFE(reg, n_features_to_select=0.0833)
    sfs1 = sfs1.fit(x['train'], y['train'])
    
    X_train_sele = sfs1.transform(x['train'])
    X_test_sele = sfs1.transform(x['test'])
    
    reg.fit(X_train_sele, y['train'])
    
    prediction = reg.predict(X_train_sele)
    tn, fp, fn, tp = confusion_matrix(y['train'], prediction).ravel()
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    recall = tp/(tp+fn)
    precision = tp/(tp+fp)
    specificity = tn/(fp+tn)
    f1_sc = f1_score(y['train'], prediction)
    matthews_cor = matthews_corrcoef(y['train'], prediction)

    print('Training\naccuracy: ', accuracy, '\nrecall', recall, '\nprecision', precision, '\nspecificity', specificity, '\nf1_score', f1_sc, '\nmatthews_cor', matthews_cor)

    prediction = reg.predict(X_test_sele)
    tn, fp, fn, tp = confusion_matrix(y['test'], prediction).ravel()
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    recall = tp/(tp+fn)
    precision = tp/(tp+fp)
    specificity = tn/(fp+tn)
    f1_sc = f1_score(y['test'], prediction)
    matthews_cor = matthews_corrcoef(y['test'], prediction)

    print('Test\naccuracy: ', accuracy, '\nrecall', recall, '\nprecision', precision, '\nspecificity', specificity, '\nf1_score', f1_sc, '\nmatthews_cor', matthews_cor)

    print('KNN'+typ)
    
    knn = KNeighborsClassifier()
    knn.fit(x['train'], y['train'])
    
    prediction = knn.predict(x['train'])
    tn, fp, fn, tp = confusion_matrix(y['train'], prediction).ravel()
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    recall = tp/(tp+fn)
    precision = tp/(tp+fp)
    specificity = tn/(fp+tn)
    f1_sc = f1_score(y['train'], prediction)
    matthews_cor = matthews_corrcoef(y['train'], prediction)

    print('Training\naccuracy: ', accuracy, '\nrecall', recall, '\nprecision', precision, '\nspecificity', specificity, '\nf1_score', f1_sc, '\nmatthews_cor', matthews_cor)

    prediction = knn.predict(x['test'])
    tn, fp, fn, tp = confusion_matrix(y['test'], prediction).ravel()
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    recall = tp/(tp+fn)
    precision = tp/(tp+fp)
    specificity = tn/(fp+tn)
    f1_sc = f1_score(y['test'], prediction)
    matthews_cor = matthews_corrcoef(y['test'], prediction)

    print('Test\naccuracy: ', accuracy, '\nrecall', recall, '\nprecision', precision, '\nspecificity', specificity, '\nf1_score', f1_sc, '\nmatthews_cor', matthews_cor)
    

    print('SELECTED')

    knn.fit(X_train_sele, y['train'])
    
    prediction = knn.predict(X_train_sele)
    tn, fp, fn, tp = confusion_matrix(y['train'], prediction).ravel()
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    recall = tp/(tp+fn)
    precision = tp/(tp+fp)
    specificity = tn/(fp+tn)
    f1_sc = f1_score(y['train'], prediction)
    matthews_cor = matthews_corrcoef(y['train'], prediction)

    print('Training\naccuracy: ', accuracy, '\nrecall', recall, '\nprecision', precision, '\nspecificity', specificity, '\nf1_score', f1_sc, '\nmatthews_cor', matthews_cor)

    prediction = knn.predict(X_test_sele)
    tn, fp, fn, tp = confusion_matrix(y['test'], prediction).ravel()
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    recall = tp/(tp+fn)
    precision = tp/(tp+fp)
    specificity = tn/(fp+tn)
    f1_sc = f1_score(y['test'], prediction)
    matthews_cor = matthews_corrcoef(y['test'], prediction)

    print('Test\naccuracy: ', accuracy, '\nrecall', recall, '\nprecision', precision, '\nspecificity', specificity, '\nf1_score', f1_sc, '\nmatthews_cor', matthews_cor)

FOREST_vessel
Training
accuracy:  0.9957173447537473 
recall 0.9956896551724138 
precision 0.9956896551724138 
specificity 0.9957446808510638 
f1_score 0.9956896551724138 
matthews_cor 0.9914343360234776
Test
accuracy:  0.8142857142857143 
recall 0.8 
precision 0.8235294117647058 
specificity 0.8285714285714286 
f1_score 0.8115942028985507 
matthews_cor 0.6288281455225324
REGRESTION_vessel
Training
accuracy:  0.9978586723768736 
recall 0.9956896551724138 
precision 1.0 
specificity 1.0 
f1_score 0.9978401727861772 
matthews_cor 0.9957261828483867
Test
accuracy:  0.8571428571428571 
recall 0.8285714285714286 
precision 0.8787878787878788 
specificity 0.8857142857142857 
f1_score 0.8529411764705883 
matthews_cor 0.7154547587901781
SELECTED
Training
accuracy:  0.9957173447537473 
recall 0.9956896551724138 
precision 0.9956896551724138 
specificity 0.9957446808510638 
f1_score 0.9956896551724138 
matthews_cor 0.9914343360234776
Test
accuracy:  0.8428571428571429 
recall 0.8 
precision 0.87